## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torchvision.io import read_image
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader,ConcatDataset
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch import optim
import datetime
from sklearn.preprocessing import StandardScaler

## Lectura de dataset train y test

In [2]:
train = pd.read_csv('../Dataset/train.csv')
test = pd.read_csv('../Dataset/test.csv')

train.head()

,bathrooms,bedrooms,area,zipcode,bathroom_image,kitchen_image,bedroom_image,frontal_image,price
0,5.0,5,3816,92880,285_bathroom.jpg,285_kitchen.jpg,285_bedroom.jpg,285_frontal.jpg,589900
1,2.0,2,1440,92276,348_bathroom.jpg,348_kitchen.jpg,348_bedroom.jpg,348_frontal.jpg,106000
2,3.0,4,1625,93510,441_bathroom.jpg,441_kitchen.jpg,441_bedroom.jpg,441_frontal.jpg,639000
3,3.0,4,2454,93510,422_bathroom.jpg,422_kitchen.jpg,422_bedroom.jpg,422_frontal.jpg,5858000
4,4.5,4,4038,92677,150_bathroom.jpg,150_kitchen.jpg,150_bedroom.jpg,150_frontal.jpg,1795000


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bathrooms       428 non-null    float64
 1   bedrooms        428 non-null    int64  
 2   area            428 non-null    int64  
 3   zipcode         428 non-null    int64  
 4   bathroom_image  428 non-null    object 
 5   kitchen_image   428 non-null    object 
 6   bedroom_image   428 non-null    object 
 7   frontal_image   428 non-null    object 
 8   price           428 non-null    int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 30.2+ KB


## Preparar Dataset

In [29]:
class CustomImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df                # DataFrame que contiene los datos
        self.transform = transform  # Transformaciones opcionales para las imágenes

        # --> Seleccionar las columnas que se van a estandarizar
        self.numeric_features = ['bedrooms', 'bathrooms', 'area', 'zipcode']
        
        # --> Inicializar el estandarizador y ajustarlo a los datos
        self.scaler = StandardScaler()
        self.df[self.numeric_features] = self.scaler.fit_transform(self.df[self.numeric_features])

    def __len__(self):
        return len(self.df)

    # --> Método para obtener una muestra del conjunto de datos dado un índice
    def __getitem__(self, idx):
        # --> Obtener las rutas de las imágenes para baño, cocina, dormitorio y frontal
        # bathroom_img_path = self.df.loc[idx, 'bathroom_image']
        # kitchen_img_path = self.df.loc[idx, 'kitchen_image']
        # bedroom_img_path = self.df.loc[idx, 'bedroom_image']
        frontal_img_path = self.df.loc[idx, 'frontal_image']
        
        # --> Leer las imágenes desde las rutas especificadas y convertirlas a tensores float
        # bathroom_image = read_image(bathroom_img_path).float()
        # kitchen_image = read_image(kitchen_img_path).float()
        # bedroom_image = read_image(bedroom_img_path).float()
        frontal_image = read_image(f"../Dataset/Houses-dataset/Houses Dataset/{frontal_img_path}").float()
        
        # --> Obtener los valores de las características numéricas estandarizadas desde el DataFrame
        bathrooms = torch.tensor(self.df.loc[idx, 'bathrooms'], dtype=torch.float32)
        bedrooms = torch.tensor(self.df.loc[idx, 'bedrooms'], dtype=torch.float32)
        area = torch.tensor(self.df.loc[idx, 'area'], dtype=torch.float32)
        zipcode = torch.tensor(self.df.loc[idx, 'zipcode'], dtype=torch.float32)
        price = torch.tensor(self.df.loc[idx, 'price'], dtype=torch.float32)
        
        # --> Aplicar transformaciones a las imágenes si se proporcionan
        if self.transform:
            # bathroom_image = self.transform(bathroom_image)
            # kitchen_image = self.transform(kitchen_image)
            # bedroom_image = self.transform(bedroom_image)
            frontal_image = self.transform(frontal_image)
        
        # --> Concatenar las características numéricas estandarizadas en un solo tensor
        numeric_features = torch.hstack((bathrooms, bedrooms, area, zipcode))
        
        # --> Crear un diccionario de las imágenes
        # images = {
        #     'bathroom': bathroom_image,
        #     'kitchen': kitchen_image,
        #     'bedroom': bedroom_image,
        #     'frontal': frontal_image
        # }
        
        # Devolver las imágenes, las características numéricas y el precio
        return frontal_image, numeric_features, price

## Aumentar datos

In [30]:
data1=CustomImageDataset(df=train,transform=transforms.Compose([transforms.Resize(256),
                                                                transforms.RandomCrop((224,224)),
                                                                transforms.Normalize(
                                                                    (58.0583, 55.1679, 52.9831),
                                                                    (85.9875, 82.3628, 80.8718))
                                                                    ]))
data2=CustomImageDataset(df=train,transform=transforms.Compose([transforms.Resize(256),
                                                                transforms.CenterCrop((224,224)),
                                                                transforms.ColorJitter(
                                                                    brightness=0.6, contrast=1.3, saturation=1.1, hue=.2),
                                                                transforms.Normalize(
                                                                    (58.0583, 55.1679, 52.9831),
                                                                    (85.9875, 82.3628, 80.8718)),
                                                                    ]))
data3=CustomImageDataset(df=train,transform=transforms.Compose([transforms.Resize(256),
                                                                transforms.CenterCrop((224,224)),
                                                                transforms.ColorJitter(
                                                                    brightness=1.3, contrast=.5, saturation=.5),
                                                                transforms.RandomHorizontalFlip(p=1),
                                                                transforms.Normalize(
                                                                    (58.0583, 55.1679, 52.9831),
                                                                    (85.9875, 82.3628, 80.8718))
                                                                    ]))
data4=CustomImageDataset(df=train,transform=transforms.Compose([transforms.Resize(256),
                                                                transforms.CenterCrop((224,224)),
                                                                transforms.ColorJitter(
                                                                    brightness=.5, contrast=.5, saturation=.5),
                                                                transforms.RandomVerticalFlip(p=1),
                                                                transforms.Normalize(
                                                                    (58.0583, 55.1679, 52.9831),
                                                                    (85.9875, 82.3628, 80.8718))
                                                                    ]))
data5=CustomImageDataset(df=train,transform=transforms.Compose([transforms.Resize((400,400)),
                                                                transforms.RandomRotation(degrees = 160),
                                                                transforms.CenterCrop((224,224)),
                                                                transforms.Normalize(
                                                                    (58.0583, 55.1679, 52.9831),
                                                                    (85.9875, 82.3628, 80.8718))
                                                                    ]))
data6=CustomImageDataset(df=train,transform=transforms.Compose([transforms.Resize(400),
                                                                transforms.RandomRotation(degrees = 45),
                                                                transforms.CenterCrop((224,224)),
                                                                transforms.RandomHorizontalFlip(p=1),
                                                                transforms.Normalize(
                                                                    (58.0583, 55.1679, 52.9831),
                                                                    (85.9875, 82.3628, 80.8718))
                                                                    ]))
data7=CustomImageDataset(df=train,transform=transforms.Compose([transforms.RandomVerticalFlip(p=1),
                                                                transforms.RandomHorizontalFlip(p=1),
                                                                transforms.Resize((224,224)),
                                                                transforms.ColorJitter(
                                                                    brightness=.5, contrast=.5, saturation=.5),
                                                                transforms.Normalize(
                                                                    (58.0583, 55.1679, 52.9831),
                                                                    (85.9875, 82.3628, 80.8718))
                                                                    ]))

In [31]:
# Crear un conjunto de datos combinado
conjunto_datos = ConcatDataset((data1,data2,data3,data4,data5,data6,data7))


In [32]:
# Determinar el tamaño de los conjuntos de entrenamiento y validación
valor_train = int(0.8*len(conjunto_datos))
valor_val = len(conjunto_datos)-valor_train

print(f"Train: {valor_train} Val: {valor_val}")


Train: 2396 Val: 600


In [33]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y validación

train_data,val_data=torch.utils.data.random_split(ConcatDataset((data1,data2,data3,data4,data5,data6,data7)),[valor_train,valor_val])

del data1,data2,data3,data4,data5,data6,data7

In [34]:
# Crear los dataloaders

batch_size=64
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

## Red neuronal

In [42]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.image_features_ = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),

            nn.Conv2d(16, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),

            nn.Conv2d(128, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),

            nn.Conv2d(256, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(128, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.numeric_features_ = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64, 64*3),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64*3, 64*3*3),
            nn.ReLU(inplace=True),
        )
        self.combined_features_ = nn.Sequential(
            nn.Linear(64*3*3*2, 64*3*3*2*2),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64*3*3*2*2, 64*3*3*2),
            nn.ReLU(inplace=True),
            nn.Linear(64*3*3*2, 64),
            nn.Linear(64, 1),
        )

    def forward(self, x,y):
        x = self.image_features_(x)
        x=x.view(-1, 64*3*3)
        y=self.numeric_features_(y)
        z=torch.cat((x,y),1)
        z=self.combined_features_(z)
        return z.squeeze(1)

In [43]:
# Definir el dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

# Definir el modelo, la función de pérdida y el optimizador
model = NeuralNetwork().to(device)              # Modelo de red neuronal mandarlo al dispositivo (GPU)
optimizer=optim.Adam(model.parameters(),1e-3)   # Adam optimizer para optimización
loss_fn=nn.MSELoss()                            # Mean Squared Error Loss para regresión

Using cpu device


## Ciclos de entrenamiento y pruebas

In [44]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for imgs,numeric_features, price in dataloader:
            imgs=imgs.to(device)
            numeric_features=numeric_features.to(device)
            price = price.to(device)
            pred = model(imgs,numeric_features)
            test_loss += loss_fn(pred, price).item()
            correct += (pred.argmax(1) == price).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [45]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
    for epoch in range(1,n_epochs+1):
        loss_train = 0.0
        for imgs,numeric_features, price in train_loader:
            imgs=imgs.to(device)
            numeric_features=numeric_features.to(device)
            price = price.to(device)
            output=model(imgs, numeric_features)

            
            loss=loss_fn(output, price)
            
            #L2 Regularization
            l2_lambda=0.001
            l2_norm=sum(p.pow(2).sum() for p in model.parameters())
            loss=loss+l2_lambda*l2_norm

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))
            test_loop(dataloader=val_loader, model=model, loss_fn=loss_fn)

## Entrenamiento

In [46]:
training_loop(
n_epochs = 100,
optimizer = optimizer,
model = model,
loss_fn = loss_fn,
train_loader = train_dataloader,
val_loader=val_dataloader)

KeyboardInterrupt: 